In [5]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import math
import os

def load_data():
    
    # Load city coordinates
    cities_df = pd.read_csv('cities_data.csv')
    coords = {}
    for _, row in cities_df.iterrows():
        coords[row['city']] = (row['x'], row['y'])
    
    # Load area assignments
    areas_df = pd.read_csv('areas_data.csv')
    N1 = set(areas_df[areas_df['area'] == 'N1']['city'].tolist())
    N2 = set(areas_df[areas_df['area'] == 'N2']['city'].tolist())
    N3 = set(areas_df[areas_df['area'] == 'N3']['city'].tolist())
    
    return coords, N1, N2, N3

def define_constraints(coords):
    """Define fixed problem constraints"""
    N = set(coords.keys())
    
    # Policy sets (fixed constraints)
    K1 = {"Konin", "Kalisz"} & N           # at least one
    K2 = {"Bytom", "Sosnowiec", "Katowice"} & N  # at most one
    K3 = {"Wodzisław Śląski", "Jastrzębie-Zdrój"} & N  # at most one
    K4 = {"Poznań", "Zielona Góra", "Leszno"} & N      # at least one
    
    # Mandatory edges (fixed infrastructure)
    mandatory_edges = [("Radom", "Kielce"), ("Gliwice", "Opole")]
    
    # Mandatory cities (fixed requirements)
    mandatory_cities = ["Radom", "Kielce", "Gliwice", "Opole"]
    
    return K1, K2, K3, K4, mandatory_edges, mandatory_cities

def euclid(a, b, coords):
    """Calculate Euclidean distance between two cities"""
    ax, ay = coords[a]
    bx, by = coords[b]
    return math.hypot(ax - bx, ay - by)

def solve_telecom_path():
    """Main optimization function"""
    
    # Load variable data from files
    coords, N1, N2, N3 = load_data()
    
    # Define fixed constraints
    K1, K2, K3, K4, mandatory_edges, mandatory_cities = define_constraints(coords)
    
    # Build set of all cities
    N = set(coords.keys())
    
    # Verify data integrity
    assert N1 | N2 | N3 <= N, "Area sets must be subset of N"
    assert (N1 & N2) == set() and (N1 & N3) == set() and (N2 & N3) == set(), "Areas must be disjoint"
    
    # Precompute distances (no self-arcs)
    d = {(i, j): euclid(i, j, coords) for i in N for j in N if i != j}
    
    # Create optimization model
    m = gp.Model("Telecom_Path_SW_Poland")
    m.setParam('OutputFlag', 1)  # Enable output
    
    # Big-M for MTZ ordering
    M = len(N)
    
    # Decision variables
    x = m.addVars(d.keys(), vtype=GRB.BINARY, name="x")  # Arc selection
    y = m.addVars(N, vtype=GRB.BINARY, name="y")        # City inclusion
    u = m.addVars(N, lb=0.0, ub=M, vtype=GRB.CONTINUOUS, name="u")  # Position ordering
    
    # Objective: minimize total cable length
    m.setObjective(gp.quicksum(d[i, j] * x[i, j] for (i, j) in d), GRB.MINIMIZE)
    
    # Constraints
    
    # 1) Start/End constraints
    m.addConstr(gp.quicksum(x["Warsaw", j] for j in N if j != "Warsaw") == 1, "start_out")
    m.addConstr(gp.quicksum(x[i, "Warsaw"] for i in N if i != "Warsaw") == 0, "start_in")
    m.addConstr(gp.quicksum(x[i, "Jelenia Góra"] for i in N if i != "Jelenia Góra") == 1, "end_in")
    m.addConstr(gp.quicksum(x["Jelenia Góra", j] for j in N if j != "Jelenia Góra") == 0, "end_out")
    
    # Flow conservation for intermediate nodes
    for j in N - {"Warsaw", "Jelenia Góra"}:
        m.addConstr(gp.quicksum(x[i, j] for i in N if i != j) == y[j], f"in_{j}")
        m.addConstr(gp.quicksum(x[j, k] for k in N if k != j) == y[j], f"out_{j}")
    
    # City inclusion for endpoints
    m.addConstr(y["Warsaw"] == 1, "y_warsaw")
    m.addConstr(y["Jelenia Góra"] == 1, "y_jg")
    
    # 2) Mandatory cities (fixed business requirements)
    for city in mandatory_cities:
        if city in N:
            m.addConstr(y[city] == 1, f"mandatory_{city}")
    
    # 3) Area coverage counts
    m.addConstr(gp.quicksum(y[i] for i in N1) == 4, "area1_count")
    m.addConstr(gp.quicksum(y[i] for i in N2) == 5, "area2_count")
    m.addConstr(gp.quicksum(y[i] for i in N3) == 5, "area3_count")
    
    # 4) Policy constraints (fixed business rules)
    if K1: m.addConstr(gp.quicksum(y[i] for i in K1) >= 1, "policy_K1")
    if K2: m.addConstr(gp.quicksum(y[i] for i in K2) <= 1, "policy_K2")
    if K3: m.addConstr(gp.quicksum(y[i] for i in K3) <= 1, "policy_K3")
    if K4: m.addConstr(gp.quicksum(y[i] for i in K4) >= 1, "policy_K4")
    
    # 5) No bidirectional connections
    for i in N:
        for j in N:
            if i < j and (i, j) in x and (j, i) in x:
                m.addConstr(x[i, j] + x[j, i] <= 1, f"nobidir_{i}_{j}")
    
    # 6) Edge-activation implies node-activation
    for (i, j) in d:
        m.addConstr(x[i, j] <= y[i], f"x_le_yi_{i}_{j}")
        m.addConstr(x[i, j] <= y[j], f"x_le_yj_{i}_{j}")
    
    # 7) Mandatory existing infrastructure edges
    for (a, b) in mandatory_edges:
        if (a in N) and (b in N) and (a, b) in x:
            m.addConstr(x[a, b] == 1, f"mandatory_edge_{a}_{b}")
    
    # 8) MTZ subtour elimination
    m.addConstr(u["Warsaw"] == 1, "u_start")
    for i in N - {"Warsaw"}:
        m.addConstr(u[i] >= y[i], f"u_lb_{i}")
        m.addConstr(u[i] <= M * y[i], f"u_ub_{i}")
    
    for i in N:
        for j in N:
            if i != j and (i, j) in x:
                m.addConstr(u[i] - u[j] + M * x[i, j] <= M - 1, f"mtz_{i}_{j}")
    
    # Solve the model
    print("Optimizing telecommunications path...")
    m.optimize()
    
    # Extract and display results
    if m.status == GRB.OPTIMAL:
        print("\n" + "="*60)
        print("OPTIMAL SOLUTION FOUND")
        print("="*60)
        
        # Extract solution
        total_length = m.objVal
        selected_nodes = [i for i in N if y[i].X > 0.5]
        
        # Build successor map and reconstruct path
        succ = {i: None for i in N}
        selected_edges = []
        for (i, j), var in x.items():
            if var.X > 0.5:
                succ[i] = j
                selected_edges.append((i, j, d[i, j]))
        
        # Reconstruct path starting from Warsaw
        path = ["Warsaw"]
        while succ[path[-1]] is not None:
            path.append(succ[path[-1]])
            if len(path) > len(N):  # Safety check
                break
        
        # Required outputs
        print(f"\nLength of the main cable: {total_length:.3f} units")
        print(f"\nCities connected to the main cable (in order from Warsaw):")
        for i, city in enumerate(path, 1):
            print(f"  {i:2d}. {city}")
        
        # Additional information
        print(f"\nTotal cities selected: {len(selected_nodes)}")
        print(f"Path segments:")
        for i in range(len(path) - 1):
            from_city, to_city = path[i], path[i + 1]
            segment_length = d[from_city, to_city]
            print(f"  {from_city} -> {to_city}: {segment_length:.3f} units")
        
        # Verify area constraints
        print(f"\nArea distribution:")
        print(f"  Area N1: {len([c for c in selected_nodes if c in N1])} cities")
        print(f"  Area N2: {len([c for c in selected_nodes if c in N2])} cities") 
        print(f"  Area N3: {len([c for c in selected_nodes if c in N3])} cities")
        
        return path, total_length
        
    else:
        print(f"\nOptimization failed. Status: {m.status}")
        if m.status == GRB.INFEASIBLE:
            print("The problem is infeasible - no solution satisfies all constraints.")
        elif m.status == GRB.UNBOUNDED:
            print("The problem is unbounded.")
        else:
            print("Check Gurobi documentation for status code meaning.")
        return None, None

if __name__ == "__main__":
    # Check if data files exist
    required_files = ['cities_data.csv', 'areas_data.csv']
    for file in required_files:
        if not os.path.exists(file):
            print(f"Error: Required data file '{file}' not found!")
            print("Please ensure all data files are in the same directory as this script.")
            exit(1)
    
    # Solve the optimization problem
    path, length = solve_telecom_path()

Set parameter OutputFlag to value 1
Optimizing telecommunications path...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 23.6.0 23H626)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2393 rows, 702 columns and 6693 nonzeros
Model fingerprint: 0xfde8ce83
Variable types: 26 continuous, 676 integer (676 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [2e+00, 6e+01]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 1637 rows and 161 columns
Presolve time: 0.02s
Presolved: 756 rows, 541 columns, 3135 nonzeros
Variable types: 22 continuous, 519 integer (519 binary)
Found heuristic solution: objective 447.9609584
Found heuristic solution: objective 341.1510616

Root relaxation: objective 1.327592e+02, 125 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl U

In [10]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import math
import numpy as np
import sys
import os

# Set UTF-8 encoding for the environment
os.environ['PYTHONIOENCODING'] = 'utf-8'
if sys.platform == 'win32':
    import locale
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

def load_data(filename='cities_data.csv'):
    """Load city data from CSV file"""
    # Read the CSV file
    cities_df = pd.read_csv(filename)
    
    # Extract coordinates and capacities
    coords = {}
    capacities = {}
    
    for _, row in cities_df.iterrows():
        city = row['city']
        coords[city] = (row['x'], row['y'])
        capacities[city] = row['capacity_tb']
    
    return coords, capacities

def euclid(city_a, city_b, coords):
    """Calculate Euclidean distance between two cities"""
    x1, y1 = coords[city_a]
    x2, y2 = coords[city_b]
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def clean_name_for_constraint(name):
    """Convert city name to ASCII for constraint names"""
    # Replace Polish characters with ASCII equivalents
    replacements = {
        'ą': 'a', 'ć': 'c', 'ę': 'e', 'ł': 'l', 'ń': 'n', 
        'ó': 'o', 'ś': 's', 'ź': 'z', 'ż': 'z',
        'Ą': 'A', 'Ć': 'C', 'Ę': 'E', 'Ł': 'L', 'Ń': 'N',
        'Ó': 'O', 'Ś': 'S', 'Ź': 'Z', 'Ż': 'Z',
        ' ': '_', '-': '_'
    }
    clean = name
    for polish, ascii_char in replacements.items():
        clean = clean.replace(polish, ascii_char)
    return clean

def solve_task2_switching_stations():
    """
    Solve Task 2: Connect remaining cities to switching stations
    Using the main line from Task 1 solution
    """
    
    # Load city data
    coords, capacities = load_data('cities_data.csv')
    
    # Main line cities from Task 1 solution (from your output)
    main_line_cities = [
        "Warsaw", "Radom", "Kielce", "Kraków", "Bielsko",
        "Jastrzębie-Zdrój", "Gliwice", "Opole", "Sieradz",
        "Konin", "Kalisz", "Leszno", "Legnica", "Jelenia Góra"
    ]
    
    # Define sets
    N = set(coords.keys())  # All cities
    main_line_set = set(main_line_cities)
    R = N - main_line_set  # Remaining cities not on main line
    S = {"Warsaw", "Kielce", "Opole"}  # Switching stations
    
    print("="*60)
    print("TASK 2: CONNECTING REMAINING CITIES TO SWITCHING STATIONS")
    print("="*60)
    print(f"\nTotal cities: {len(N)}")
    print(f"Cities on main line: {len(main_line_set)}")
    print(f"Remaining cities to connect: {len(R)}")
    print(f"\nRemaining cities: {sorted(R)}")
    print(f"Switching stations: {sorted(S)}")
    
    # Parameters
    alpha = 1000  # Cost per coordinate unit per TB/s (EUR)
    beta = 200    # Maximum capacity per cable (TB/s)
    
    # Switching station capacities (TB/s)
    C = {
        "Warsaw": 800,
        "Kielce": 1200,
        "Opole": 400
    }
    
    # City capacity requirements for remaining cities
    c = {city: capacities[city] for city in R}
    
    # Verify all remaining cities have capacity requirements
    print("\nCapacity requirements for remaining cities:")
    for city in sorted(R):
        print(f"  {city}: {c[city]} TB/s")
    
    # Calculate distances from remaining cities to switching stations
    d = {}
    for i in R:
        for j in S:
            d[i, j] = euclid(i, j, coords)
    
    # Create Gurobi model
    m = gp.Model("Task2_Switching_Stations")
    m.setParam('OutputFlag', 0)  # Completely disable Gurobi output to avoid encoding issues
    
    # Enable sensitivity analysis
    m.setParam('SolutionLimit', 1)
    
    # Create mapping for clean variable names
    R_clean = {city: clean_name_for_constraint(city) for city in R}
    S_clean = {station: clean_name_for_constraint(station) for station in S}
    
    # Decision variables: Flow from city i to station j (continuous, >= 0)
    # Use clean names for variables too
    Q = {}
    for i in R:
        for j in S:
            Q[i, j] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, 
                              name=f"Q_{R_clean[i]}_{S_clean[j]}")
    
    # Objective: Minimize total cost of laying copper cables
    m.setObjective(
        gp.quicksum(alpha * d[i, j] * Q[i, j] for i in R for j in S),
        GRB.MINIMIZE
    )
    
    # Constraints
    
    # 1. Demand Satisfaction: Each city's requirement must be met exactly
    demand_constrs = {}
    for i in R:
        clean_i = clean_name_for_constraint(i)
        demand_constrs[i] = m.addConstr(
            gp.quicksum(Q[i, j] for j in S) == c[i],
            f"demand_{clean_i}"
        )
    
    # 2. Cable Capacity Limits: Maximum 200 TB/s per cable
    cable_constrs = {}
    for i in R:
        for j in S:
            clean_i = clean_name_for_constraint(i)
            clean_j = clean_name_for_constraint(j)
            cable_constrs[i, j] = m.addConstr(
                Q[i, j] <= beta, 
                f"cable_cap_{clean_i}_{clean_j}"
            )
    
    # 3. Switching Station Capacity
    station_constrs = {}
    for j in S:
        clean_j = clean_name_for_constraint(j)
        station_constrs[j] = m.addConstr(
            gp.quicksum(Q[i, j] for i in R) <= C[j],
            f"station_cap_{clean_j}"
        )
    
    # 4. Shannon's Law: Signal degradation with distance
    shannon_constrs = {}
    for i in R:
        for j in S:
            clean_i = clean_name_for_constraint(i)
            clean_j = clean_name_for_constraint(j)
            shannon_limit = 2 * math.pi * min(35, 70 - d[i, j])
            if shannon_limit > 0:
                shannon_constrs[i, j] = m.addConstr(
                    Q[i, j] <= shannon_limit, 
                    f"shannon_{clean_i}_{clean_j}"
                )
            else:
                # Distance too large, no connection possible
                shannon_constrs[i, j] = m.addConstr(
                    Q[i, j] == 0, 
                    f"shannon_zero_{clean_i}_{clean_j}"
                )
    
    # 5. Łódź Special Constraint: Maximum 40% on any single cable
    lodz_constrs = {}
    if "Łódź" in R:
        for j in S:
            clean_j = clean_name_for_constraint(j)
            lodz_constrs[j] = m.addConstr(
                Q["Łódź", j] <= 0.4 * c["Łódź"],
                f"lodz_limit_{clean_j}"
            )
    
    # Optimize
    print("\n" + "-"*40)
    print("Optimizing...")
    print("-"*40)
    
    # Suppress Gurobi output during optimization
    m.setParam('OutputFlag', 0)
    m.optimize()
    
    # Print optimization status
    if m.status == GRB.OPTIMAL:
        print(f"Optimization successful! Objective: {m.objVal:.2f}")
    else:
        print(f"Optimization status: {m.status}")
    
    # Check solution status
    if m.status != GRB.OPTIMAL:
        print(f"\nOptimization failed. Status: {m.status}")
        if m.status == GRB.INFEASIBLE:
            print("Problem is infeasible. Computing IIS...")
            m.computeIIS()
            m.write("task2_iis.ilp")
            print("IIS written to task2_iis.ilp")
        return None
    
    # Extract solution
    print("\n" + "="*60)
    print("OPTIMAL SOLUTION FOUND")
    print("="*60)
    
    total_cost = m.objVal
    
    # Question 1: Total costs
    print(f"\n1. TOTAL COST TO CONNECT REMAINING CITIES:")
    print(f"   {total_cost:,.2f} EUR")
    
    # Question 2: Poznań and Wałbrzych capacity
    print(f"\n2. CAPACITY INSTALLED FOR POZNAŃ AND WAŁBRZYCH:")
    print("-"*50)
    
    for city in ["Poznań", "Wałbrzych"]:
        if city in R:
            print(f"\n{city}:")
            total_installed = 0
            for j in S:
                flow = Q[city, j].X
                if flow > 0.001:
                    cost = alpha * d[city, j] * flow
                    print(f"  -> {j}: {flow:.2f} TB/s (Cost: {cost:,.2f} EUR)")
                    total_installed += flow
            print(f"  Total installed: {total_installed:.2f} TB/s")
            print(f"  Required: {c[city]:.2f} TB/s")
        else:
            print(f"{city}: On main line (not a remaining city)")
    
    # Question 3: Kielce utilization
    print(f"\n3. KIELCE SWITCHING STATION UTILIZATION:")
    print("-"*50)
    kielce_flow = sum(Q[i, "Kielce"].X for i in R)
    kielce_utilization = (kielce_flow / C["Kielce"]) * 100
    print(f"  Total capacity: {C['Kielce']} TB/s")
    print(f"  Used capacity: {kielce_flow:.2f} TB/s")
    print(f"  Utilization: {kielce_utilization:.2f}%")
    
    # Additional analysis
    print(f"\n4. DETAILED CONNECTION SUMMARY:")
    print("-"*50)
    
    # Station utilization summary
    print("\nSwitching Station Utilization:")
    for j in S:
        total_flow = sum(Q[i, j].X for i in R)
        utilization = (total_flow / C[j]) * 100
        print(f"  {j}: {total_flow:.2f}/{C[j]} TB/s ({utilization:.2f}%)")
    
    # All connections
    print("\nAll Connections (City -> Station: Flow):")
    connections = []
    for i in sorted(R):
        city_connections = []
        for j in S:
            flow = Q[i, j].X
            if flow > 0.001:
                city_connections.append(f"{j}({flow:.1f})")
                connections.append({
                    'City': i,
                    'Station': j,
                    'Flow_TB_s': flow,
                    'Distance': d[i, j],
                    'Cost_EUR': alpha * d[i, j] * flow
                })
        if city_connections:
            print(f"  {i}: {', '.join(city_connections)}")
    
    # Shannon's Law analysis
    print("\n5. SHANNON'S LAW IMPACT:")
    print("-"*50)
    shannon_limited = []
    for i in R:
        for j in S:
            shannon_limit = 2 * math.pi * min(35, 70 - d[i, j])
            if Q[i, j].X > 0.001 and Q[i, j].X >= shannon_limit - 0.1:
                shannon_limited.append((i, j, Q[i, j].X, shannon_limit))
    
    if shannon_limited:
        print("Connections limited by Shannon's Law:")
        for city, station, flow, limit in shannon_limited:
            print(f"  {city} -> {station}: Flow={flow:.2f}, Limit={limit:.2f} TB/s")
    else:
        print("  No connections are limited by Shannon's Law")
    
    # Łódź special constraint check
    if "Łódź" in R:
        print("\n6. ŁÓDŹ SPECIAL CONSTRAINT (40% max per cable):")
        print("-"*50)
        for j in S:
            flow = Q["Łódź", j].X
            if flow > 0.001:
                percentage = (flow / c["Łódź"]) * 100
                print(f"  Łódź -> {j}: {flow:.2f} TB/s ({percentage:.1f}% of total)")
    
    # Sensitivity Analysis
    print("\n7. COMPREHENSIVE SENSITIVITY ANALYSIS:")
    print("="*50)
    
    # 1. Shadow Prices (Dual Values) for Constraints
    print("\n7.1 SHADOW PRICES (Dual Values):")
    print("-"*40)
    
    # Demand constraints
    print("\n  A. Demand Constraints:")
    print("     (Shadow price = marginal cost change per TB/s demand change)")
    for city in sorted(R):
        if city in demand_constrs:
            constr = demand_constrs[city]
            shadow = constr.Pi
            rhs_low = constr.SARHSLow if hasattr(constr, 'SARHSLow') else float('-inf')
            rhs_up = constr.SARHSUp if hasattr(constr, 'SARHSUp') else float('inf')
            print(f"\n    {city}:")
            print(f"      Shadow Price: {shadow:,.2f} EUR/TB")
            print(f"      Current RHS: {c[city]:.2f} TB/s")
            print(f"      Valid Range: [{rhs_low:.2f}, {rhs_up:.2f}] TB/s")
    
    # Station capacity constraints
    print("\n  B. Station Capacity Constraints:")
    print("     (Shadow price = value of additional capacity)")
    for station in sorted(S):
        if station in station_constrs:
            constr = station_constrs[station]
            shadow = constr.Pi
            rhs_low = constr.SARHSLow if hasattr(constr, 'SARHSLow') else float('-inf')
            rhs_up = constr.SARHSUp if hasattr(constr, 'SARHSUp') else float('inf')
            current_usage = sum(Q[i, station].X for i in R)
            print(f"\n    {station}:")
            print(f"      Shadow Price: {shadow:,.2f} EUR/TB")
            print(f"      Current Capacity: {C[station]:.2f} TB/s")
            print(f"      Current Usage: {current_usage:.2f} TB/s")
            print(f"      Valid Range: [{rhs_low:.2f}, {rhs_up:.2f}] TB/s")
    
    # Łódź special constraints
    if "Łódź" in R and lodz_constrs:
        print("\n  C. Łódź Special Constraints (40% limit):")
        for station in S:
            if station in lodz_constrs:
                constr = lodz_constrs[station]
                shadow = constr.Pi
                rhs_low = constr.SARHSLow if hasattr(constr, 'SARHSLow') else float('-inf')
                rhs_up = constr.SARHSUp if hasattr(constr, 'SARHSUp') else float('inf')
                print(f"\n    Łódź -> {station}:")
                print(f"      Shadow Price: {shadow:,.2f} EUR/TB")
                print(f"      Current Limit: {0.4 * c['Łódź']:.2f} TB/s")
                print(f"      Valid Range: [{rhs_low:.2f}, {rhs_up:.2f}] TB/s")
    
    # 2. Reduced Costs and Objective Coefficient Ranges
    print("\n7.2 REDUCED COSTS & OBJECTIVE COEFFICIENT RANGES:")
    print("-"*40)
    print("  (Reduced cost = cost penalty for forcing non-basic variable into solution)")
    
    variables_with_rc = []
    for i in R:
        for j in S:
            var = Q[i, j]
            rc = var.RC
            obj_coef = alpha * d[i, j]  # Current objective coefficient
            obj_low = var.SAObjLow if hasattr(var, 'SAObjLow') else float('-inf')
            obj_up = var.SAObjUp if hasattr(var, 'SAObjUp') else float('inf')
            
            if abs(rc) > 0.001 or var.X < 0.001:  # Non-basic or at bound
                variables_with_rc.append({
                    'city': i,
                    'station': j,
                    'flow': var.X,
                    'reduced_cost': rc,
                    'obj_coef': obj_coef,
                    'obj_low': obj_low,
                    'obj_up': obj_up
                })
    
    if variables_with_rc:
        print("\n  Variables with non-zero reduced costs or at bounds:")
        for v in sorted(variables_with_rc, key=lambda x: abs(x['reduced_cost']), reverse=True):
            print(f"\n    Q[{v['city']} -> {v['station']}]:")
            print(f"      Current Flow: {v['flow']:.2f} TB/s")
            print(f"      Reduced Cost: {v['reduced_cost']:,.2f} EUR")
            print(f"      Current Obj Coef: {v['obj_coef']:,.2f} EUR/TB")
            print(f"      Obj Coef Range: [{v['obj_low']:,.2f}, {v['obj_up']:,.2f}] EUR/TB")
    else:
        print("  All variables at optimal values with zero reduced cost")
    
    # 3. Basis Status Information
    print("\n7.3 BASIS STATUS SUMMARY:")
    print("-"*40)
    
    basic_count = 0
    nonbasic_lower = 0
    nonbasic_upper = 0
    superbasic = 0
    
    for i in R:
        for j in S:
            var = Q[i, j]
            if hasattr(var, 'VBasis'):
                if var.VBasis == GRB.BASIC:
                    basic_count += 1
                elif var.VBasis == GRB.NONBASIC_LOWER:
                    nonbasic_lower += 1
                elif var.VBasis == GRB.NONBASIC_UPPER:
                    nonbasic_upper += 1
                elif var.VBasis == GRB.SUPERBASIC:
                    superbasic += 1
    
    print(f"  Basic variables: {basic_count}")
    print(f"  Non-basic at lower bound: {nonbasic_lower}")
    print(f"  Non-basic at upper bound: {nonbasic_upper}")
    print(f"  Superbasic variables: {superbasic}")
    
    # 4. Active Constraints Summary
    print("\n7.4 ACTIVE CONSTRAINTS (Binding at optimum):")
    print("-"*40)
    
    active_constraints = []
    
    # Check all constraint types
    for city in R:
        if city in demand_constrs:
            constr = demand_constrs[city]
            if abs(constr.Slack) < 0.001:
                active_constraints.append(f"Demand_{city}")
    
    for station in S:
        if station in station_constrs:
            constr = station_constrs[station]
            if abs(constr.Slack) < 0.001:
                active_constraints.append(f"Station_Capacity_{station}")
    
    for (i, j) in shannon_constrs:
        constr = shannon_constrs[i, j]
        if abs(constr.Slack) < 0.001:
            active_constraints.append(f"Shannon_{i}_{j}")
    
    if "Łódź" in R:
        for station in lodz_constrs:
            constr = lodz_constrs[station]
            if abs(constr.Slack) < 0.001:
                active_constraints.append(f"Lodz_limit_{station}")
    
    if active_constraints:
        print("  Active (binding) constraints:")
        for constr_name in active_constraints:
            print(f"    - {constr_name}")
    else:
        print("  No constraints are binding at the optimum")
    
    # 5. Sensitivity Summary Table
    print("\n7.5 KEY SENSITIVITY INSIGHTS:")
    print("-"*40)
    
    # Find most sensitive demand constraint
    max_shadow = 0
    most_sensitive_city = None
    for city in R:
        if city in demand_constrs:
            shadow = abs(demand_constrs[city].Pi)
            if shadow > max_shadow:
                max_shadow = shadow
                most_sensitive_city = city
    
    if most_sensitive_city:
        print(f"  Most expensive city to serve: {most_sensitive_city}")
        print(f"    (Shadow price: {demand_constrs[most_sensitive_city].Pi:,.2f} EUR/TB)")
    
    # Find bottleneck station
    bottleneck_station = None
    max_utilization = 0
    for station in S:
        usage = sum(Q[i, station].X for i in R)
        utilization = usage / C[station]
        if utilization > max_utilization:
            max_utilization = utilization
            bottleneck_station = station
    
    if bottleneck_station:
        print(f"\n  Most utilized station: {bottleneck_station}")
        print(f"    (Utilization: {max_utilization*100:.1f}%)")
        if station_constrs[bottleneck_station].Pi > 0.001:
            print(f"    Shadow price: {station_constrs[bottleneck_station].Pi:,.2f} EUR/TB")
            print(f"    => Adding 1 TB capacity would save {station_constrs[bottleneck_station].Pi:,.2f} EUR")
    
    # Save results to CSV
    df = pd.DataFrame(connections)
    df.to_csv('task2_results.csv', index=False)
    print("\n8. RESULTS SAVED TO FILES:")
    print("  - task2_results.csv (connection details)")
    
    # Export model for further analysis
    m.write("task2_model.lp")
    m.write("task2_solution.sol")
    print("  - task2_model.lp (model file)")
    print("  - task2_solution.sol (solution file)")
    
    print("\n" + "="*60)
    print("TASK 2 COMPLETED SUCCESSFULLY")
    print("="*60)
    
    return {
        'model': m,
        'total_cost': total_cost,
        'connections': connections,
        'variables': Q
    }

if __name__ == "__main__":
    # Run Task 2 optimization
    results = solve_task2_switching_stations()

TASK 2: CONNECTING REMAINING CITIES TO SWITCHING STATIONS

Total cities: 26
Cities on main line: 14
Remaining cities to connect: 12

Remaining cities: ['Bytom', 'Czestochowa', 'Katowice', 'Piotrków', 'Poznań', 'Skierniewice', 'Sosnowiec', 'Wałbrzych', 'Wodzisław Śląski', 'Wrocław', 'Zielona Góra', 'Łódź']
Switching stations: ['Kielce', 'Opole', 'Warsaw']

Capacity requirements for remaining cities:
  Bytom: 163 TB/s
  Czestochowa: 278 TB/s
  Katowice: 263 TB/s
  Piotrków: 139 TB/s
  Poznań: 251 TB/s
  Skierniewice: 197 TB/s
  Sosnowiec: 143 TB/s
  Wałbrzych: 129 TB/s
  Wodzisław Śląski: 119 TB/s
  Wrocław: 133 TB/s
  Zielona Góra: 121 TB/s
  Łódź: 317 TB/s

----------------------------------------
Optimizing...
----------------------------------------
Optimization successful! Objective: 57301741.69

OPTIMAL SOLUTION FOUND

1. TOTAL COST TO CONNECT REMAINING CITIES:
   57,301,741.69 EUR

2. CAPACITY INSTALLED FOR POZNAŃ AND WAŁBRZYCH:
--------------------------------------------------

